## Examples of Data fetching using CCXT 

In [1]:
from datetime import datetime, timezone
import configparser
import json
import requests
import ccxt
import time
from pprint import pprint 
import pandas as pd
from urllib.parse import urljoin 
from utils import ccxt_utils, cq_utils   
import pickle

config = configparser.ConfigParser()
config.read('../config.ini')

BINANCE_SYMBOL = 'BTC/USDT'
COPRO_SYMBOL = 'BTC/USD'

BINANCE_API_KEY = config['BINANCE']['API_KEY']
BINANCE_SECRET_KEY = config['BINANCE']['SECRET_KEY']
BINANCE_API_URL = config['BINANCE']['API_URL']

COPRO_API_KEY = config['COINBASE']['API_KEY']
COPRO_SECRET_KEY = config['COINBASE']['SECRET_KEY']
COPRO_API_URL = config['COINBASE']['API_URL']

In [2]:
print('CCXT Version: ', ccxt.__version__)

binance = ccxt.binance({
    'options': {
        'adjustForTimeDifference': True,
        # 'defaultType': 'future'
    },
    'rateLimit': 10000,
    'enableRateLimit': True,
    # 'verbose': True,
    'apiKey': BINANCE_API_KEY,
    'secret': BINANCE_SECRET_KEY,
})

binance_futures = ccxt.binance({
    'options': {
        'adjustForTimeDifference': True,
        'defaultType': 'future'
    },
    'rateLimit': 10000,
    'enableRateLimit': True,
    # 'verbose': True,
    'apiKey': BINANCE_API_KEY,
    'secret': BINANCE_SECRET_KEY,
})

copro = ccxt.coinbasepro({
    'options': {
        'adjustForTimeDifference': True,
        'defaultType': 'future'
    },
    'rateLimit': 10000,
    'enableRateLimit': True,
    # 'verbose': True,
    'apiKey': COPRO_API_KEY,
    'secret': COPRO_SECRET_KEY,
})

CCXT Version:  1.45.42


In [3]:
binance.id, binance_futures.id, copro.id

('binance', 'binance', 'coinbasepro')

In [4]:
since = '2021-06-21 00:00:00'
since_timestamp = binance.parse8601(since)
print(since_timestamp)

a_data = ccxt_utils.fetch_ohlcv(BINANCE_SYMBOL, binance, timeframe='1m', start=since, end=None)
b_data = ccxt_utils.fetch_ohlcv(BINANCE_SYMBOL, binance_futures, timeframe='1m', start=since, end=None)
c_data = ccxt_utils.fetch_ohlcv(COPRO_SYMBOL, copro, timeframe='1m', start=since, end=None)


1624233600000
Data fetching (binance): from  2021-06-21T00:00:00.000Z - to 2021-06-24T01:29:45.402Z
Data fetching (binance): from  2021-06-21T08:20:00.000Z - to 2021-06-24T01:29:45.402Z
Data fetching (binance): from  2021-06-21T16:40:00.000Z - to 2021-06-24T01:29:45.402Z
Data fetching (binance): from  2021-06-22T01:00:00.000Z - to 2021-06-24T01:29:45.402Z
Data fetching (binance): from  2021-06-22T09:20:00.000Z - to 2021-06-24T01:29:45.402Z
Data fetching (binance): from  2021-06-22T17:40:00.000Z - to 2021-06-24T01:29:45.402Z
Data fetching (binance): from  2021-06-23T02:00:00.000Z - to 2021-06-24T01:29:45.402Z
Data fetching (binance): from  2021-06-23T10:20:00.000Z - to 2021-06-24T01:29:45.402Z
Data fetching (binance): from  2021-06-23T18:40:00.000Z - to 2021-06-24T01:29:45.402Z
Fetching finished.
Data fetching (binance): from  2021-06-21T00:00:00.000Z - to 2021-06-24T01:31:25.564Z
Data fetching (binance): from  2021-06-21T08:20:00.000Z - to 2021-06-24T01:31:25.564Z
Data fetching (binanc

In [5]:
a_data, b_data, c_data 

({'status': 200,
  'data': [[1624233600000, 35600.17, 35611.11, 35529.73, 35576.53, 52.186536],
   [1624233660000, 35578.92, 35581.79, 35510.73, 35523.99, 48.084236],
   [1624233720000, 35525.75, 35717.0, 35525.75, 35692.69, 129.77635],
   [1624233780000, 35694.57, 35750.0, 35649.21, 35669.46, 64.674063],
   [1624233840000, 35669.45, 35669.46, 35507.57, 35511.98, 56.100678],
   [1624233900000, 35511.97, 35562.84, 35458.19, 35467.65, 65.857022],
   [1624233960000, 35467.64, 35490.02, 35455.0, 35465.6, 49.868475],
   [1624234020000, 35465.61, 35486.98, 35436.1, 35486.83, 33.138414],
   [1624234080000, 35486.52, 35494.71, 35378.77, 35399.99, 72.539641],
   [1624234140000, 35399.99, 35412.21, 35355.45, 35404.74, 114.53281],
   [1624234200000, 35405.54, 35459.17, 35377.01, 35393.31, 42.337897],
   [1624234260000, 35393.3, 35419.31, 35369.54, 35408.78, 31.181787],
   [1624234320000, 35408.77, 35436.84, 35395.83, 35433.38, 38.009656],
   [1624234380000, 35433.38, 35470.71, 35428.14, 35444.72,

In [7]:
a_data_df = ccxt_utils.ohlcv_to_df(a_data['data'])
b_data_df = ccxt_utils.ohlcv_to_df(b_data['data'])
b_data_df

,open,high,low,close,volume
datetime,,,,,
2021-06-21 00:00:00,35588.63,35590.02,35516.83,35570.48,705.065
2021-06-21 00:01:00,35569.42,35572.42,35504.12,35521.43,510.509
2021-06-21 00:02:00,35522.03,35700.00,35522.02,35680.20,1102.520
2021-06-21 00:03:00,35680.19,35746.28,35630.75,35656.84,808.413
2021-06-21 00:04:00,35656.85,35656.85,35500.10,35504.62,914.691
...,...,...,...,...,...
2021-06-24 01:28:00,33109.58,33124.81,33056.21,33110.77,456.049
2021-06-24 01:29:00,33107.59,33160.00,33073.47,33076.75,343.097
2021-06-24 01:30:00,33076.75,33129.40,33076.74,33109.45,310.805


In [ ]:
# with open('data/20200101-binance_futures-1m.pkl', 'wb') as f: 
#     pickle.dump(b_data, f, pickle.HIGHEST_PROTOCOL) 

In [8]:
merged_data = ccxt_utils.merge_price_signal(a_data_df, b_data_df)
merged_data

,open_x,high_x,low_x,close_x,volume_x,open_y,high_y,low_y,close_y,volume_y
datetime,,,,,,,,,,
2021-06-21 00:00:00,35600.17,35611.11,35529.73,35576.53,52.186536,35588.63,35590.02,35516.83,35570.48,705.065
2021-06-21 00:01:00,35578.92,35581.79,35510.73,35523.99,48.084236,35569.42,35572.42,35504.12,35521.43,510.509
2021-06-21 00:02:00,35525.75,35717.00,35525.75,35692.69,129.776350,35522.03,35700.00,35522.02,35680.20,1102.520
2021-06-21 00:03:00,35694.57,35750.00,35649.21,35669.46,64.674063,35680.19,35746.28,35630.75,35656.84,808.413
2021-06-21 00:04:00,35669.45,35669.46,35507.57,35511.98,56.100678,35656.85,35656.85,35500.10,35504.62,914.691
...,...,...,...,...,...,...,...,...,...,...
2021-06-24 01:27:00,33127.89,33134.07,33081.63,33119.27,54.188373,33119.30,33121.14,33056.64,33109.58,525.422
2021-06-24 01:28:00,33120.77,33136.23,33070.01,33121.71,50.379468,33109.58,33124.81,33056.21,33110.77,456.049
2021-06-24 01:29:00,33116.72,33173.10,33083.99,33085.17,54.653258,33107.59,33160.00,33073.47,33076.75,343.097
